In [3]:
!python --version


Python 2.7.13 :: Continuum Analytics, Inc.


In [252]:
import os 
__file__='Consumer_Complaints.csv'
dir_path = os.path.dirname(os.path.realpath(__file__))
print(dir_path)

/Users/ishmaelamin


In [322]:
import json


import spacy
import pandas as pd
import itertools as it

nlp = spacy.load('en')



In [259]:
import os
import codecs

data_directory = os.path.join('.','Documents','Northwestern_University','2017_Fall','Predict_453','Predict_453_data')

customer_filepath = os.path.join(data_directory,
                                   'reviews_Grocery_and_Gourmet_Food_5.json')

with codecs.open(customer_filepath, encoding='utf_8') as f:
    first_customer_record = f.readline() 




In [260]:
print(first_customer_record)


{"reviewerID": "A1VEELTKS8NLZB", "asin": "616719923X", "reviewerName": "Amazon Customer", "helpful": [0, 0], "reviewText": "Just another flavor of Kit Kat but the taste is unique and a bit different.  The only thing that is bothersome is the price.  I thought it was a bit expensive....", "overall": 4.0, "summary": "Good Taste", "unixReviewTime": 1370044800, "reviewTime": "06 1, 2013"}



In [275]:
df=pd.read_json(customer_filepath, lines=True)

In [276]:
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,616719923X,"[0, 0]",4,Just another flavor of Kit Kat but the taste i...,"06 1, 2013",A1VEELTKS8NLZB,Amazon Customer,Good Taste,1370044800
1,616719923X,"[0, 1]",3,I bought this on impulse and it comes from Jap...,"05 19, 2014",A14R9XMZVJ6INB,amf0001,"3.5 stars, sadly not as wonderful as I had hoped",1400457600
2,616719923X,"[3, 4]",4,Really good. Great gift for any fan of green t...,"10 8, 2013",A27IQHDZFQFNGG,Caitlin,Yum!,1381190400
3,616719923X,"[0, 0]",5,"I had never had it before, was curious to see ...","05 20, 2013",A31QY5TASILE89,DebraDownSth,Unexpected flavor meld,1369008000
4,616719923X,"[1, 2]",4,I've been looking forward to trying these afte...,"05 26, 2013",A2LWK003FFMCI5,Diana X.,"Not a very strong tea flavor, but still yummy ...",1369526400


In [309]:


from gensim.models import Phrases
from gensim.models.word2vec import LineSentence



In [295]:

def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])


In [300]:
unigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'unigram_sentences_all.txt')


review_txt_filepath = os.path.join(intermediate_directory,
                                          'review_txt_filepath.txt')

Loop over the original review text while utilizing the lemmatized_sentence_corpus generator 
We will segmenting the reviews into individual sentences
normalizing the text
write this data back out to a new file (unigram_sentences_all)
normalized sentence per line.
use this data for learning our phrase models.

In [301]:
#df.to_csv(r'c:\data\pandas.txt', header=None, index=None, sep=' ', mode='a')
df['reviewText'].to_csv(review_txt_filepath, index=None, sep=' ', mode='a')

In [302]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus(review_txt_filepath):
            f.write(sentence + '\n')



CPU times: user 16min 21s, sys: 17.3 s, total: 16min 39s
Wall time: 16min 59s


In [303]:


unigram_sentences = LineSentence(unigram_sentences_filepath)


    


In [304]:


for unigram_sentence in it.islice(unigram_sentences, 230, 240):
    print u' '.join(unigram_sentence)
    print u''



traditional medicinal breathe easy be an amazing tea

a couple of time -PRON- have have asthma attack -PRON- be clear power have work

-PRON- be a delicious spicy mix of egyptian licorice peppermint ginger and eucalyptus

-PRON- be the drinkable vaposteam&it benefit be incredible

traditional medicinals be base in beautiful sebastopol california and -PRON- healing herb come from -PRON- garden

-PRON- buy this to compare -PRON- against yogi tea 's 34;breathe deep&#34 tea because of the low cost

-PRON- be pleasant enough but -PRON- prefer the yogi tea better

this brand taste good but seem to be less effective in clear a cough or congestion than yogi tea

if -PRON- wife drink a cup of this tea when -PRON- feel an attack come on -PRON- often help -PRON- avoid the attack at all if -PRON- be out and -PRON- have an attack drink this tea have lessen the effect and help -PRON- recover more quickly

-PRON- have try other tincture and tea but -PRON- blend from tm have be wonderful



Next, we will learn a phrase model that will connect words into 2 word phrases, like mountain climber.


In [305]:


bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model_all')



In [306]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 1 == 1:

    bigram_model = Phrases(unigram_sentences)

    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

CPU times: user 32.7 s, sys: 1.23 s, total: 33.9 s
Wall time: 34.4 s


In [307]:
bigram_sentences_filepath = os.path.join(intermediate_directory,
                                         'bigram_sentences_all.txt')

In [310]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

CPU times: user 1min 22s, sys: 2.43 s, total: 1min 24s
Wall time: 1min 25s


In [311]:


bigram_sentences = LineSentence(bigram_sentences_filepath)



In [312]:
for bigram_sentence in it.islice(bigram_sentences, 230, 240):
    print u' '.join(bigram_sentence)
    print u''

traditional_medicinal breathe easy be an amazing tea

a couple of time -PRON- have have asthma_attack -PRON- be clear power have work

-PRON- be a delicious spicy mix of egyptian_licorice peppermint ginger and eucalyptus

-PRON- be the drinkable vaposteam&it benefit be incredible

traditional_medicinals be base in beautiful sebastopol california and -PRON- healing herb come from -PRON- garden

-PRON- buy this to compare -PRON- against yogi_tea 's 34;breathe deep&#34 tea because of the low cost

-PRON- be pleasant enough but -PRON- prefer the yogi_tea better

this brand taste good but seem to be less effective in clear a cough or congestion than yogi_tea

if -PRON- wife drink a cup of this tea when -PRON- feel an attack come on -PRON- often help -PRON- avoid the attack at all if -PRON- be out and -PRON- have an attack drink this tea have lessen the effect and help -PRON- recover more quickly

-PRON- have try other tincture and tea but -PRON- blend from tm have be wonderful



Next we will train for 3 word phrases

In [313]:


trigram_model_filepath = os.path.join(intermediate_directory,
                                      'trigram_model_all')



In [314]:


%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 1 == 1:

    trigram_model = Phrases(bigram_sentences)

    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)



CPU times: user 36.5 s, sys: 1.66 s, total: 38.2 s
Wall time: 39.3 s


Apply our trained second-order phrase model to our first-order transformed sentences, 
write the results out to a new file, and 
explore a few of the second-order transformed sentences.

In [315]:


trigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'trigram_sentences_all.txt')



In [316]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            
            trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            
            f.write(trigram_sentence + '\n')

CPU times: user 1min 21s, sys: 2.6 s, total: 1min 23s
Wall time: 1min 24s


In [317]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [318]:
for trigram_sentence in it.islice(trigram_sentences, 230, 240):
    print u' '.join(trigram_sentence)
    print u''

traditional_medicinal breathe easy be an amazing tea

a couple of time -PRON- have have asthma_attack -PRON- be clear power have work

-PRON- be a delicious spicy mix of egyptian_licorice peppermint ginger and eucalyptus

-PRON- be the drinkable vaposteam&it benefit be incredible

traditional_medicinals be base in beautiful sebastopol california and -PRON- healing herb come from -PRON- garden

-PRON- buy this to compare -PRON- against yogi_tea 's 34;breathe deep&#34 tea because of the low cost

-PRON- be pleasant enough but -PRON- prefer the yogi_tea better

this brand taste good but seem to be less effective in clear a cough or congestion than yogi_tea

if -PRON- wife drink a cup of this tea when -PRON- feel an attack come on -PRON- often help -PRON- avoid the attack at all if -PRON- be out and -PRON- have an attack drink this tea have lessen the effect and help -PRON- recover more quickly

-PRON- have try other tincture and tea but -PRON- blend from tm have be wonderful



write the transformed text out to a new file, with one review per line

In [319]:


trigram_reviews_filepath = os.path.join(intermediate_directory,
                                        'trigram_transformed_reviews_all.txt')



In [330]:
from spacy.en.language_data import STOP_WORDS


#import spacy.en.English.Defaults.stop_words


In [331]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                      batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in STOP_WORDS]
            
            # write the transformed review as a line in the new file
            trigram_review = u' '.join(trigram_review)
            f.write(trigram_review + '\n')

CPU times: user 18min 36s, sys: 18.7 s, total: 18min 55s
Wall time: 19min 12s


<ls>Read in each business record and convert it to a Python dict<ls/>
<ls>Filter out business records that aren't about restaurants (i.e., not in the "Restaurant" category)<ls/>
<ls>Create a frozenset of the business IDs for restaurants, which we'll use in the next step<ls/>


Phrase Modeling 
https://radimrehurek.com/gensim/models/phrases.html

In [332]:
print u'Original:' + u'\n'

for review in it.islice(line_review(review_txt_filepath), 11, 12):
    print review

print u'----' + u'\n'
print u'Transformed:' + u'\n'

with codecs.open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 11, 12):
        print review

Original:

"These are definitely THE BEST candy bar out there!  I just wish I was able to find them in a local store instead of having to buy them online!"

----

Transformed:

definitely best candy_bar -PRON- wish -PRON- able find -PRON- local_store instead buy -PRON- online



Grammatical structure has been removed: capitalization, articles/conjuctions, punctuation, spaciing, etc.
Multi-word concepts are present

Next we proceed with higher-level modeling



# Latent Dirichlet Allocation (LDA)

LDA is Unsupervided Learning method that applies the MLE observing documents within corpus.  

We next capture latent structure and organization within the documents that have meansingful human interprestation.


In [333]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import cPickle as pickle

First, we learn the full vocabulary of the corpus to be modeled

In [334]:


trigram_dictionary_filepath = os.path.join(intermediate_directory,
                                           'trigram_dict_all.dict')



In [335]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if 1 == 1:

    trigram_reviews = LineSentence(trigram_reviews_filepath)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

CPU times: user 12.6 s, sys: 317 ms, total: 12.9 s
Wall time: 13 s


LDA utilized Bag-of-Words model (BOW)

In [336]:
trigram_bow_filepath = os.path.join(intermediate_directory,
                                    'trigram_bow_corpus_all.mm')

In [337]:


def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)



In [338]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if 1 == 1:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_reviews_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

CPU times: user 31 s, sys: 920 ms, total: 31.9 s
Wall time: 32.3 s



We are not ready to learn our topic model from the reviews

In [339]:
lda_model_filepath = os.path.join(intermediate_directory, 'lda_model_all')

In [340]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if 1 == 1:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=50,
                           id2word=trigram_dictionary,
                           workers=3)
    
    lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

CPU times: user 2min 22s, sys: 33.8 s, total: 2min 56s
Wall time: 3min 11s


Topic model is trained. 

Each topic is represented as a mixture of tokens. Next, we analyze the results for patterns.

In [342]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print u'{:20} {}'.format(u'term', u'frequency') + u'\n'

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print u'{:20} {:.3f}'.format(term, round(frequency, 3))

In [343]:


explore_topic(topic_number=0)



term                 frequency

oatmeal              0.035
cinnamon             0.030
use                  0.020
try                  0.014
flavor               0.012
drink                0.010
work                 0.010
sweet                0.010
sugar                0.009
quaker               0.008
product              0.008
brownie              0.008
tea                  0.008
smoothie             0.007
think                0.007
tart                 0.006
buy                  0.006
find                 0.006
little               0.006
time                 0.006
great                0.006
add                  0.005
love                 0.005
gift_basket          0.005
definitely           0.005


This topic seems to relate to Quaker oatmeal, brownies smootie, tea, etc.  This implies that a variety of items are discussed.  We might assumee that products are being reviewed as part of a gift basket.

In [358]:


topic_names = {0: u'breakfast',
               1: u'food',
               2: u'ingredients',
               3: u'spices',
               4: u'sweeteners',
               5: u'brands',
               6: u'desserts',
               7: u'baked goods',
               8: u'tea & coffee',
               9: u'purchases',
               10: u'discover',
               11: u'opinion',
               12: u'satisfaction',
               13: u'artificial flavors',
               14: u'healthy',
               15: u'nutritious',
               16: u'diet',
               17: u'exercise'}

In [359]:
topic_names_filepath = os.path.join(intermediate_directory, 'topic_names.pkl')

with open(topic_names_filepath, 'w') as f:
    pickle.dump(topic_names, f)

In [360]:


LDAvis_data_filepath = os.path.join(intermediate_directory, 'ldavis_prepared')



In [374]:
#!pip install /Users/ishmaelamin/Downloads/pyLDAvis-master.zip -U
#import pyLDAvis
pyLDAvis = reload(pyLDAvis)

In [375]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                              trigram_dictionary)

    with open(LDAvis_data_filepath, 'w') as f:
        pickle.dump(LDAvis_prepared, f)
        
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath) as f:
    LDAvis_prepared = pickle.load(f)

/Users/ishmaelamin/anaconda/envs/python2/lib/python2.7/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


CPU times: user 7min 8s, sys: 7.04 s, total: 7min 15s
Wall time: 4min 29s


In [376]:


pyLDAvis.display(LDAvis_prepared)



Principal Component chart: The left visual shows the distance between the topics and their relationship with each other. 
The Marginal topic distribution is a measure of the frequency that the word occurse withing the corpus

The Bar Chart: shows the top terms found in the corpus.  A terms' saliency is a measure of the frequency and relavence of the word within the corpus.

Lamda parameter close to 1 will rank the word phrase according to their probability within the topic.  
Lamda parameter close to 0 will rank term based on the uniquenss of the term within the topic. Terms that are unique to this topic.  
The rank can be applied on a scale of lamda parameter from 0 to 1.

This is an interactive chart.


I would have expected more variety in the comments.  I am have to investigate if the whole corpus was processed correctly.



In [377]:
def get_sample_review(review_number):
    """
    retrieve a particular review index
    from the reviews file and return it
    """
    
    return list(it.islice(line_review(review_txt_filepath),
                          review_number, review_number+1))[0]

In [387]:
def lda_description(review_text, min_topic_freq=0.05):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """
    
    # parse the review text with spaCy
    parsed_review = nlp(review_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_review = [token.lemma_ for token in parsed_review
                      if not punct_space(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_review = bigram_model[unigram_review]
    trigram_review = trigram_model[bigram_review]
    
    # remove any remaining stopwords
    trigram_review = [term for term in trigram_review
                      if not term in STOP_WORDS]
    
    # create a bag-of-words representation
    review_bow = trigram_dictionary.doc2bow(trigram_review)
    
    # create an LDA representation
    review_lda = lda[review_bow]
    
    # sort with the most highly related topics first
    review_lda = sorted(review_lda, key=lambda (topic_number, freq): -freq)
    
    for topic_number, freq in review_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print '{:9} {}'.format(topic_names[topic_number],
                                round(freq, 3))


In [388]:


sample_review = get_sample_review(50)
print sample_review



"I don't know about the medicinal aspects of this tea, but the flavor is downright scrumptious.  I love licorice root but have high blood pressure, so I'm not guzzling down box after box each week.  A few cups a week doesn't touch my BP, so I keep it to that.  The directions say to steep it for 10-15 minutes, but I'm far too impatient for that.  4-6 minutes is all I can wait after smelling that lovely aroma.  Traditional Medicinals puts out some really good teas.  I just wish they weren't so proud of 'em.  ChaChing!  Thanks, TM. Now try to work on that pricing for us, will ya?  Danke."



In [389]:
lda_description(sample_review)

exercise  0.372


KeyError: 21

In [390]:
sample_review = get_sample_review(100)
print sample_review

"So many coffee enthusiasts come back to Blue Mountain time and time again as one of the finest coffees you can find in the world.  This is so delicious it won't last long."



In [391]:


lda_description(sample_review)



KeyError: 47

In [392]:


from gensim.models import Word2Vec

trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = os.path.join(intermediate_directory, 'word2vec_model_all')



In [396]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the word2vec model yourself.

if 1 == 1:

    # initiate the model and perform the first epoch of training
    ##vec_model.train(sentences, total_examples=self.corpus_count, epochs=self.iter).
    food2vec = Word2Vec(trigram_sentences, size=100, window=5, min_count=20, sg=1, workers=4)
    
    food2vec.save(word2vec_filepath)

    # perform another 11 epochs of training
    for i in range(1,12):
        #model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
        food2vec.train(trigram_sentences, total_examples=food2vec.corpus_count, epochs=food2vec.iter)
        food2vec.save(word2vec_filepath)
        
# load the finished model from disk
food2vec = Word2Vec.load(word2vec_filepath)
food2vec.init_sims()

print u'{} training epochs so far.'.format(food2vec.train_count)

12 training epochs so far.
CPU times: user 1h 18min 6s, sys: 50.6 s, total: 1h 18min 57s
Wall time: 25min 41s


In [398]:
print u'{:,} terms in the food2vec vocabulary.'.format(len(food2vec.wv.vocab))

13,898 terms in the food2vec vocabulary.


In [402]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in food2vec.wv.vocab.iteritems()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda (term, index, count): -count)

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the food2vec vectors as data,
# and the terms as row labels
#food2vec.syn0norm = (food2vec.syn0 / sqrt((food2vec.syn0 ** 2).sum(-1))[..., newaxis]).astype(REAL)

word_vectors = pd.DataFrame(food2vec.wv.syn0norm[term_indices, :],
                            index=ordered_terms)


word_vectors

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
-PRON-,0.103464,0.214275,-0.036296,-0.093744,-0.031990,-0.254631,0.086142,-0.130130,-0.152948,-0.031386,...,-0.007230,-0.077964,-0.144106,0.008075,0.163143,-0.068571,0.105961,0.172593,-0.025987,-0.076230
be,0.009404,0.123408,-0.067369,0.087582,0.019341,-0.156604,0.152755,-0.092582,-0.015937,-0.076549,...,-0.186238,-0.078596,-0.109397,-0.014017,0.115655,0.138663,-0.142108,0.037907,-0.055651,-0.053248
the,0.073438,0.072044,0.137092,-0.011055,-0.178013,-0.135441,0.057931,-0.185842,0.011627,-0.193251,...,-0.054285,-0.032299,-0.121984,0.074601,-0.008395,-0.058213,0.048381,0.096342,0.045595,0.110894
a,0.192980,-0.020619,0.117851,-0.038942,-0.076734,-0.140711,0.121817,-0.050729,-0.124953,-0.014796,...,-0.063045,-0.074039,-0.095225,0.121049,0.005179,0.006013,-0.011314,0.107897,-0.049313,-0.031500
and,0.014057,0.114529,0.086363,-0.115607,-0.159813,-0.200390,0.183817,-0.082690,-0.047812,-0.045207,...,-0.092796,-0.079658,-0.184550,0.014921,0.090722,0.079196,0.054692,0.129923,-0.037924,0.050909
to,0.011197,0.136873,-0.109386,-0.124814,-0.246888,-0.064133,0.134008,-0.081284,-0.148826,0.009026,...,-0.093978,-0.064777,-0.056119,0.036091,0.087626,-0.050129,0.095620,0.188416,0.014261,-0.108176
of,0.036471,-0.018254,0.142261,-0.009875,-0.096762,-0.094140,0.129298,-0.044763,0.031199,-0.090588,...,-0.179942,-0.014973,-0.052439,-0.049185,-0.027385,0.084579,0.006657,0.092311,0.185604,-0.032361
not,0.093838,0.231738,-0.119534,-0.020923,-0.096990,-0.059245,0.036880,-0.184307,0.006516,0.015420,...,0.123346,-0.100212,-0.080451,0.015032,0.035747,-0.030319,0.065669,0.081775,-0.021904,-0.009322
this,0.039833,0.097129,0.084552,-0.083787,0.045793,-0.151961,0.049826,-0.121883,-0.129581,-0.029976,...,-0.118329,0.055689,-0.167676,0.085460,0.113030,-0.063941,0.195239,0.020281,0.073394,-0.012624
have,-0.016882,0.209162,0.044835,-0.147776,-0.034448,-0.080820,0.158642,-0.085778,-0.046809,-0.135129,...,0.011454,-0.082690,-0.105247,0.039479,0.103894,0.143626,0.017939,-0.037270,-0.015595,-0.193403


#Our model has learned a quantitative vector representation for each term, as expected.
#Now we can  look up related words and phrases for a given term of interest.

In [404]:


def get_related_terms(token, topn=10):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in food2vec.most_similar(positive=[token], topn=topn):

        print u'{:20} {}'.format(word, round(similarity, 3))



In [405]:


get_related_terms(u'pink_himalayan_salt')



himalayan_salt       0.568
trace_mineral        0.509
seeds                0.49
sea_salt             0.488
salt                 0.465
celtic_sea_salt      0.456
stunning             0.456
stevia_select        0.455
himalayan            0.441
truffle              0.436


In [406]:
get_related_terms(u'steak')

hamburger            0.786
burger               0.763
chicken              0.727
chicken_breast       0.712
grill                0.71
pork                 0.701
shrimp               0.699
hot_dog              0.689
sausage              0.689
pork_chop            0.687


In [407]:
get_related_terms(u'healthy')

healthful            0.842
nutritious           0.804
wholesome            0.693
unhealthy            0.69
low_calorie          0.667
relatively_healthy   0.662
heart_healthy        0.658
junk_food            0.646
healthy_alternative  0.646
low_cal              0.633


In [410]:
get_related_terms(u'dairy')


soy                  0.758
gluten               0.707
wheat                0.656
allergy              0.645
vegan                0.636
animal               0.631
peanut_tree_nut      0.628
gluten_dairy         0.625
dairy_soy            0.619
wheat_gluten         0.608


The model has learn words that are similar to one another in the dataset.

# Analogy completion or Word Algebra



No self-respecting word2vec demo would be complete without a healthy dose of word algebra, also known as analogy completion.

In [412]:


def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = food2vec.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print term



In [413]:


word_algebra(add=[u'breakfast', u'lunch'])



dinner


In [414]:
word_algebra(add=[u'lunch', u'night'], subtract=[u'day'])

dinner


In [415]:


word_algebra(add=[u'taco', u'chinese'], subtract=[u'mexican'])



hot_dog


In [416]:
word_algebra(add=[u'bun', u'mexican'], subtract=[u'american'])

tacos


In [420]:
word_algebra(add=[u'wine', u'cheese'], subtract=[u'dairy'])

balsamic_vinegar


# Word Vector Visualization with t-SNE

In [421]:
from sklearn.manifold import TSNE

t-Distributed Stochastic Neighbor Embedding is another form of dimensionality reduction technique to assist with visualizing high-dimensional datasets

In [426]:
tsne_input = word_vectors.drop(STOP_WORDS, errors=u'ignore')
tsne_input = tsne_input.head(5000)

In [427]:


tsne_input.head()



,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
-PRON-,0.103464,0.214275,-0.036296,-0.093744,-0.031990,-0.254631,0.086142,-0.130130,-0.152948,-0.031386,...,-0.007230,-0.077964,-0.144106,0.008075,0.163143,-0.068571,0.105961,0.172593,-0.025987,-0.076230
good,-0.129162,0.080792,0.195954,-0.088178,-0.066577,-0.273265,0.091101,-0.004748,0.019156,-0.064806,...,-0.180355,0.146869,-0.182763,-0.126028,-0.046157,0.028783,0.051143,0.054894,0.027861,-0.142558
like,-0.125677,0.121277,-0.015598,-0.137479,-0.192369,-0.111764,0.010018,-0.094405,-0.004313,-0.033612,...,-0.058702,-0.022599,-0.126667,-0.149449,-0.004328,0.116005,0.136680,0.100956,-0.011452,-0.060866
taste,-0.039901,0.066158,0.124770,-0.024525,-0.120421,-0.060905,0.076230,-0.019325,-0.169421,0.089836,...,-0.009004,-0.009922,-0.047600,0.046668,0.110880,0.087204,-0.032368,0.021440,-0.085465,-0.121427
flavor,-0.122181,0.022534,0.050287,0.006600,0.024642,-0.044510,0.124649,-0.127972,-0.061184,0.080217,...,-0.122751,0.059942,-0.016743,0.099290,0.107573,-0.012871,-0.008735,0.071855,-0.019705,-0.058441


In [428]:


tsne_filepath = os.path.join(intermediate_directory,
                             u'tsne_model')

tsne_vectors_filepath = os.path.join(intermediate_directory,
                                     u'tsne_vectors.npy')



In [429]:
%%time

if 1 == 1:
    
    tsne = TSNE()
    tsne_vectors = tsne.fit_transform(tsne_input.values)
    
    with open(tsne_filepath, 'w') as f:
        pickle.dump(tsne, f)

    pd.np.save(tsne_vectors_filepath, tsne_vectors)
    
with open(tsne_filepath) as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsne_input.index),
                            columns=[u'x_coord', u'y_coord'])

CPU times: user 1min 54s, sys: 10.6 s, total: 2min 5s
Wall time: 2min 5s


In [430]:


tsne_vectors.head()



,x_coord,y_coord
-PRON-,13.804665,58.039757
good,-8.780178,48.352554
like,19.252802,49.967598
taste,-29.765892,30.139221
flavor,-30.238018,29.280071


In [431]:
tsne_vectors[u'word'] = tsne_vectors.index

# Plotting with Bokeh

In [432]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

Loading BokehJS ...

In [434]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title=u't-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools= (u'pan, wheel_zoom, box_zoom,'
                           u'box_select, reset'),
                   active_scroll=u'wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = u'@word') )

# draw the words as circles on the plot
tsne_plot.circle(u'x_coord', u'y_coord', source=plot_data,
                 color=u'blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color=u'black')

# configure visual elements of the plot
tsne_plot.title.text_font_size = value(u'16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);